# 📦 Preprocessing and Feature Engineering in Python of the Energy and Weather datasets combineds

In this notebook, we begin the modeling phase using the cleaned and merged dataset previously generated during the EDA stage in `R`. The data includes hourly electricity demand, energy production by source, pricing, and weather observations.

## 📦 1. Preprocessing & Feature Engineering

### 🧼 1.1 Data Source & Structure

We start by:

1. Importing the cleaned dataset (`combined_clean.csv`)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype

# parse_dates=["datetime"] to ensure convert the datetime column to datetime objects
df = pd.read_csv("../data/processed/combined_clean.csv", parse_dates=["datetime"])

The dataset `combined_clean.csv` was created from a merge of two raw data sources:

- **Energy data**: Hourly electricity consumption, generation, and pricing.
- **Weather data**: Meteorological conditions across Spain’s five largest cities.

The data was preprocessed in `R` and saved to disk for use in Python.

#### 🔎 1.1.1 Cleaning Decisions (Already Applied in R)

These preprocessing steps were applied in the EDA notebook:

- Removed two features with **100% missing values**:
  - `generation_hydro_pumped_storage_aggregated`
  - `forecast_wind_offshore_eday_ahead`
- Renamed timestamp fields to `datetime` and ensured consistency.
- Merged energy and weather data using a `left_join()` on datetime.
- Extracted **time-based features**: `hour`, `wday` (weekday), and `month`.
- Exported the cleaned dataset as `data/processed/combined_clean.csv` (from the root directory of the project).

2. Verifying the structure and consistency of the dataset

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

##### 🔍 Visual Inspection: Energy Load Over Time

This plot gives an initial view of the total electricity demand over the entire time range. We expect to observe:

- Daily and seasonal patterns
- Sudden drops or spikes (potential anomalies or holidays)
- Long-term trends in consumption

This will help guide our feature engineering and baseline modeling decisions.

In [ ]:
plt.figure(figsize=(14, 4))
plt.plot(df["datetime"], df["total_load_actual"], alpha=0.7)
plt.title("Energy Load Over Time")
plt.xlabel("Date")
plt.ylabel("Load (MW)")
plt.tight_layout()
plt.show()

3. Performing additional feature engineering if needed

### ⚠️ 1.2 Missing Data Status

Although major missing columns were removed, **some NA values still remain** (as seen in `df.info()`):

- These were not removed during EDA to preserve full context for visualization.
- Now, before modeling, we should consider:
  - 🗑️ **Removing remaining NA rows**
  - 🧠 Or **imputing missing values if relevant**

👉 You can inspect this with:

In [ ]:
df.isna().sum().sort_values(ascending=False).head(10)

### ✅ 1.3 Remove Remaining Missing Values

Although only a tiny fraction of rows have missing values (less than 0.1%), we will remove them to ensure clean modeling pipelines.

- The following operation **removes all rows** that contain at least one `NaN` value.
- This preserves temporal consistency and avoids imputing potentially misleading values.

In [ ]:
# Drop all rows with at least one missing value
initial_shape = df.shape
df.dropna(inplace=True)
final_shape = df.shape

print(f"Removed {initial_shape[0] - final_shape[0]} rows with missing values.")

💡 **Note**: With over 178,000 rows, this operation removes less than 0.1% of the data and simplifies the modeling workflow.

### 📦 1.4 Encode Temporal Features

We encode the weekday (wday) and month (month) columns as integers to prepare them for machine learning models.

Since both features have a natural order (e.g., Monday to Sunday, January to December), we apply ordinal encoding instead of one-hot encoding for simplicity and interpretability.

In [ ]:
# Convert weekday (Mon–Sun) and month (Jan–Dec) to numerical values
# Ensure proper categorical order before conversion


# Define ordered categories
wday_order = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
month_order = [
    "Jan",
    "Feb",
    "Mar",
    "Apr",
    "May",
    "Jun",
    "Jul",
    "Aug",
    "Sep",
    "Oct",
    "Nov",
    "Dec",
]

# Set categorical types
df["wday"] = (
    df["wday"].astype(CategoricalDtype(categories=wday_order, ordered=True)).cat.codes
)
df["month"] = (
    df["month"].astype(CategoricalDtype(categories=month_order, ordered=True)).cat.codes
)

💡 This encoding transforms weekdays to integers from 0 (Mon) to 6 (Sun), and months from 0 (Jan) to 11 (Dec), preserving ordinal structure while simplifying downstream model input.

##### 🧠 Why We Used `pandas.api.types.CategoricalDtype` instead of `LabelEncoder`

Although scikit-learn provides tools like `LabelEncoder` to convert categorical text to numeric values, we chose to use `pandas.api.types.CategoricalDtype(...).cat.codes` for the following reasons:

- ✅ **Explicit order control**: `CategoricalDtype` allows us to manually specify the natural order of categories (e.g., Monday → Sunday), which is essential for time-based features like `wday` and `month`.
- ✅ **Cleaner integration**: Since our data is already loaded in a `pandas` DataFrame, this approach avoids the need to fit an encoder or manage external objects.
- ✅ **NaN-safe**: `pandas` preserves missing values gracefully, whereas `LabelEncoder` throws errors on `NaN`s (even tough we have eliminated `NaN` values befor this step).
- ✅ **More transparent**: The mapping between categories and codes is easily inspectable and reversible using .categories.

> In contrast, `LabelEncoder` is better suited for encoding target variables or when using fully automated pipelines with scikit-learn.

For this reason, our encoding is both more interpretable and better aligned with temporal modeling needs.

### 🔁 1.5 Lag and Rolling Features

To capture historical behavior and temporal dependencies, we create lagged and rolling window features based on `total_load_actual`.

- **Lag features** allow the model to “look back” at past values, which is crucial in time series forecasting.
- **Rolling features** help smooth short-term fluctuations and highlight trends or cycles.

These engineered features are often predictive in energy demand and similar applications.

In [ ]:
# We sort the dataframe by city_name and datetime
# to ensure that the lagged and rolling features are calculated correctly
df = df.sort_values(["city_name", "datetime"])

# Lagged features
df["load_lag_1h"] = df.groupby("city_name")["total_load_actual"].shift(1)
df["load_lag_24h"] = df.groupby("city_name")["total_load_actual"].shift(24)

# Rolling mean features
df["load_roll_24h"] = (
    df.groupby("city_name")["total_load_actual"]
    .rolling(24)
    .mean()
    .reset_index(level=0, drop=True)
)
df["load_roll_7d"] = (
    df.groupby("city_name")["total_load_actual"]
    .rolling(24 * 7)
    .mean()
    .reset_index(level=0, drop=True)
)

### 🏙️ City-aware Lag and Rolling Features

⚠️ **Important!** Since each timestamp contains one row per city, applying lag features globally would mix time dependencies across cities. This would create data leakage and distort true temporal signals.

To avoid this, we compute lag and rolling statistics **separately for each city** as below

These features are essential for capturing temporal patterns in energy demand at the city level.


As part of the feature engineering process at the last step, we obtained a lot of `NaN` values that we should handle

In [ ]:
df.isna().sum().sort_values(ascending=False).head(15)

#### ⚠️ Handling NaN Values After Lag & Rolling Features

The introduction of lagged and rolling features (e.g., `load_lag_1h`, `load_lag_24h`, `load_roll_24h`, `load_roll_7d`) naturally creates missing values at the start of each city’s time series. For example:

- `load_lag_1h` is missing the first hour,
- `load_lag_24h` and `load_roll_24h` require at least 24 hours of data,
- `load_roll_7d` needs a full 7-day (168-hour) history to compute a valid rolling mean.

Since our dataset contains hourly entries **per city**, these gaps occur **at the beginning of each city’s series**. In total, these missing values account for **835 rows**, or roughly **0.47% of the dataset**.

🔎 We choose to drop these rows rather than impute them because:

- They occur at the very start of each time series where lag/rolling windows are not defined.
- Imputing them would distort the statistical meaning of the features.
- The proportion of data lost is negligible compared to the overall dataset size.

👉 Final step:

In [ ]:
# Drop rows with any NaN values created by lag/rolling operations
df = df.dropna().reset_index(drop=True)

This ensures our dataset is clean and consistent before modeling.

### 🔚 1.6 Summary: Ready for Modeling

At this stage, we’ve completed the preprocessing and engineered useful time-based features:

- Removed noisy or fully missing columns.
- Extracted temporal variables (`hour`, `weekday`, `month`) as encoded variables.
- Generated lag features to capture short- and medium-term temporal dependencies.
- Added rolling means for signal smoothing and trend detection.
- Ensured all data is clean (dropna) and has a fresh index (reset_index).

💡 We now have a fully prepared feature matrix for modeling energy consumption.

In [ ]:
# We export the final DataFrame to a CSV file
df.to_csv("../data/processed/final_features.csv", index=False)